Adam Schmidt

CSE 411 Data Science

# FIFA World Cup Trends and Data Analysis

### Project Goal



### Data Source



In [79]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
import chart_studio.plotly as py
from operator import itemgetter
pd.options.plotting.backend = "plotly"
import json

Changing the xcell file into a csv file for ease of use.

In [4]:
FIFA_World_Cup_Stats = "FIFAWorldCupStats.xlsx"

excelFile = pd.read_excel(FIFA_World_Cup_Stats)

excelFile.to_csv ("FIFA_World_Cup_Stats.csv", index = None, header = True)

dataframe_FIFAWorldCupStats = pd.DataFrame(pd.read_csv("FIFA_World_Cup_Stats.csv"))

dataframe_FIFAWorldCupStats

,Url,Edition,Venue,City,Date and Time,Stage,Score,Home Country,Home Country Small,Home Country Scorer,Home Country Scorer Time,Away Country,Away Country Small,Away Country Scorer,Away Country Scorer Time
0,http://www.fifa.com/worldcup/matches/round=255...,2014 FIFA World Cup Brazil™,Estadio do Maracana,Rio De Janeiro (BRA),28 Jun 2014 - 17:00 Local time,Round of 16,2-0,Colombia,COL,['JAMES'],"[""28', 50'""]",Uruguay,URU,[],[]
1,http://www.fifa.com/worldcup/matches/round=255...,2014 FIFA World Cup Brazil™,Estadio Mineirao,Belo Horizonte (BRA),28 Jun 2014 - 13:00 Local time,Round of 16,1-1,Brazil,BRA,['DAVID LUIZ'],"[""18'""]",Chile,CHI,['ALEXIS'],"[""32'""]"
2,http://www.fifa.com/worldcup/matches/round=255...,2014 FIFA World Cup Brazil™,Arena Amazonia,Manaus (BRA),14 Jun 2014 - 18:00 Local time,First stage - Group D,1-2,England,ENG,['STURRIDGE'],"[""37'""]",Italy,ITA,"['MARCHISIO', 'BALOTELLI']","[""35'"", ""50'""]"
3,http://www.fifa.com/worldcup/matches/round=255...,2014 FIFA World Cup Brazil™,Estadio Castelao,Fortaleza (BRA),21 Jun 2014 - 16:00 Local time,First stage - Group G,2-2,Germany,GER,"['GÖTZE', 'KLOSE']","[""51'"", ""71'""]",Ghana,GHA,"['A. AYEW', 'A. GYAN']","[""54'"", ""63'""]"
4,http://www.fifa.com/worldcup/matches/round=255...,2014 FIFA World Cup Brazil™,Estadio Nacional,Brasilia (BRA),30 Jun 2014 - 13:00 Local time,Round of 16,2-0,France,FRA,"['POGBA', 'YOBO']","[""79'"", ""90'+2 OG""]",Nigeria,NGA,[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831,http://www.fifa.com/worldcup/matches/round=974...,2006 FIFA World Cup Germany ™_x000D_\n09 June ...,Zentralstadion,Leipzig (GER),11 Jun 2006 - 15:00 Local time,Group matches - Group C,0-1,Serbia and Montenegro,SCG,[],[],Netherlands,NED,['ROBBEN'],"[""18'""]"
832,http://www.fifa.com/worldcup/matches/round=323...,1990 FIFA World Cup Italy ™_x000D_\n08 June - ...,Luigi Ferraris,Genoa (ITA),25 Jun 1990 - 17:00 Local time,Round of 16,0-0,Republic of Ireland,IRL,[],[],Romania,ROU,[],[]
833,http://www.fifa.com/worldcup/matches/round=439...,2002 FIFA World Cup Korea/Japan ™_x000D_\n31 M...,Niigata Stadium Big Swan,Niigata (JPN),01 Jun 2002 - 15:30 Local time,Group matches - Group E,1-1,Republic of Ireland,IRL,['HOLLAND'],"[""52'""]",Cameroon,CMR,['MBOMA'],"[""39'""]"
834,http://www.fifa.com/worldcup/matches/round=337...,1994 FIFA World Cup USA ™_x000D_\n17 June - 17...,Giants Stadium,New York/New Jersey (USA),28 Jun 1994 - 12:30 Local time,Group matches - Group E,0-0,Republic of Ireland,IRL,[],[],Norway,NOR,[],[]


### Data Cleaning

This is the removal of unecessary columns and data as well as converting float columns to int. It will also check for duplicates and get rid of any. 

In [66]:
#challenge data set
#selecting columns to ignore
ignore_columns = ["Url", "Venue", "City", "Home Country Small", "Away Country Small"]
headers = [*pd.read_csv('FIFA_World_Cup_Stats.csv', nrows = 1)]
for column in ignore_columns:
    headers.remove(column)
#assigning dataset
challenge_data_set = pd.read_csv('FIFA_World_Cup_Stats.csv', usecols= headers, dtype={70: 'str'})
#Check for duplicates
challenge_data_set[challenge_data_set.duplicated(keep = "last")].shape
challenge_data_set.drop_duplicates(inplace= True)
challenge_data_set.shape



#main dataset
#selecting columns to ignore
ignore_columns2 = ["Stadium", "City", "Referee", "Assistant 1", "Assistant 2", "RoundID", "MatchID", "Home Team Initials", "Away Team Initials", "Attendance", "Win conditions", "Half-time Home Goals", "Half-time Away Goals"]
headers2 = [*pd.read_csv('WorldCupMatches.csv', nrows = 1)]
for column2 in ignore_columns2:
    headers2.remove(column2)
#assigning dataset
main_data_set_tmp = pd.read_csv('WorldCupMatches.csv', usecols= headers2, dtype={70: 'str'})
#cleaning the data set of extra rows
main_data_set_tmp[~main_data_set_tmp['Year'].isna()].shape
#changing data to int
main_data_set = main_data_set_tmp[~main_data_set_tmp["Year"].isna()].astype({"Year": "int64", "Home Team Goals": "int64", "Away Team Goals": "int64"})
#Checking if there are duplicates
main_data_set[main_data_set.duplicated(keep = "last")].shape
main_data_set.drop_duplicates(inplace= True)
main_data_set.shape

#main data set simplified
main_world_cup_info = pd.read_csv('WorldCups.csv')


main_data_set


,Year,Datetime,Stage,Home Team Name,Home Team Goals,Away Team Goals,Away Team Name
0,1930,13 Jul 1930 - 15:00,Group 1,France,4,1,Mexico
1,1930,13 Jul 1930 - 15:00,Group 4,USA,3,0,Belgium
2,1930,14 Jul 1930 - 12:45,Group 2,Yugoslavia,2,1,Brazil
3,1930,14 Jul 1930 - 14:50,Group 3,Romania,3,1,Peru
4,1930,15 Jul 1930 - 16:00,Group 1,Argentina,1,0,France
...,...,...,...,...,...,...,...
831,2014,05 Jul 2014 - 13:00,Quarter-finals,Argentina,1,0,Belgium
832,2014,29 Jun 2014 - 13:00,Round of 16,Netherlands,2,1,Mexico
833,2014,29 Jun 2014 - 17:00,Round of 16,Costa Rica,1,1,Greece
834,2014,01 Jul 2014 - 13:00,Round of 16,Argentina,1,0,Switzerland


In [43]:
challenge_data_set

,Edition,Date and Time,Stage,Score,Home Country,Home Country Scorer,Home Country Scorer Time,Away Country,Away Country Scorer,Away Country Scorer Time
0,2014 FIFA World Cup Brazil™,28 Jun 2014 - 17:00 Local time,Round of 16,2-0,Colombia,['JAMES'],"[""28', 50'""]",Uruguay,[],[]
1,2014 FIFA World Cup Brazil™,28 Jun 2014 - 13:00 Local time,Round of 16,1-1,Brazil,['DAVID LUIZ'],"[""18'""]",Chile,['ALEXIS'],"[""32'""]"
2,2014 FIFA World Cup Brazil™,14 Jun 2014 - 18:00 Local time,First stage - Group D,1-2,England,['STURRIDGE'],"[""37'""]",Italy,"['MARCHISIO', 'BALOTELLI']","[""35'"", ""50'""]"
3,2014 FIFA World Cup Brazil™,21 Jun 2014 - 16:00 Local time,First stage - Group G,2-2,Germany,"['GÖTZE', 'KLOSE']","[""51'"", ""71'""]",Ghana,"['A. AYEW', 'A. GYAN']","[""54'"", ""63'""]"
4,2014 FIFA World Cup Brazil™,30 Jun 2014 - 13:00 Local time,Round of 16,2-0,France,"['POGBA', 'YOBO']","[""79'"", ""90'+2 OG""]",Nigeria,[],[]
...,...,...,...,...,...,...,...,...,...,...
831,2006 FIFA World Cup Germany ™_x000D_\n09 June ...,11 Jun 2006 - 15:00 Local time,Group matches - Group C,0-1,Serbia and Montenegro,[],[],Netherlands,['ROBBEN'],"[""18'""]"
832,1990 FIFA World Cup Italy ™_x000D_\n08 June - ...,25 Jun 1990 - 17:00 Local time,Round of 16,0-0,Republic of Ireland,[],[],Romania,[],[]
833,2002 FIFA World Cup Korea/Japan ™_x000D_\n31 M...,01 Jun 2002 - 15:30 Local time,Group matches - Group E,1-1,Republic of Ireland,['HOLLAND'],"[""52'""]",Cameroon,['MBOMA'],"[""39'""]"
834,1994 FIFA World Cup USA ™_x000D_\n17 June - 17...,28 Jun 1994 - 12:30 Local time,Group matches - Group E,0-0,Republic of Ireland,[],[],Norway,[],[]


Seperation of desired fields to compare data to look into. 

In [50]:
main_year = main_data_set["Year"]
challenge_year = challenge_data_set["Edition"]
main_stage = main_data_set["Stage"]
challenge_stage = challenge_data_set["Stage"]
main_home_goals = main_data_set["Home Team Goals"]
main_away_goals = main_data_set["Away Team Goals"]
challenge_score = challenge_data_set["Score"]
challenge_home_country_scorer = challenge_data_set["Home Country Scorer"]
challenge_away_country_scorer = challenge_data_set["Away Country Scorer"]
main_home_country = main_data_set["Home Team Name"]
main_away_country = main_data_set["Away Team Name"]

#Simplified main data set:
simp_year = main_world_cup_info["Year"]
simp_winner = main_world_cup_info["Winner"]
simp_second = main_world_cup_info["Runners-Up"]
simp_third = main_world_cup_info["Third"]
simp_fourth = main_world_cup_info["Fourth"]

Some functions to help seperate data into specific data frames

In [31]:
def world_cup_winning_teams(year, winner):
    
    return final

In [93]:
world_cup_winners = simp_winner.value_counts()
world_cup_winners

Brazil        5
Italy         4
Germany FR    3
Uruguay       2
Argentina     2
England       1
France        1
Spain         1
Germany       1
Name: Winner, dtype: int64

In [94]:
world_cup_second = simp_second.value_counts()
world_cup_second

Argentina         3
Germany FR        3
Netherlands       3
Czechoslovakia    2
Hungary           2
Brazil            2
Italy             2
Sweden            1
Germany           1
France            1
Name: Runners-Up, dtype: int64

In [95]:
world_cup_third = simp_third.value_counts()
world_cup_third

Germany        3
Brazil         2
Sweden         2
France         2
Poland         2
USA            1
Austria        1
Chile          1
Portugal       1
Germany FR     1
Italy          1
Croatia        1
Turkey         1
Netherlands    1
Name: Third, dtype: int64

In [96]:
world_cup_fourth = simp_fourth.value_counts()
world_cup_fourth

Uruguay           3
Yugoslavia        2
Brazil            2
Austria           1
Sweden            1
Spain             1
Germany FR        1
Soviet Union      1
Italy             1
France            1
Belgium           1
England           1
Bulgaria          1
Netherlands       1
Korea Republic    1
Portugal          1
Name: Fourth, dtype: int64

In [97]:
format_finishes = pd.concat([world_cup_winners, world_cup_second, world_cup_third, world_cup_fourth], axis = 1)
format_finishes.fillna(0, inplace=True)
format_finishes 

,Winner,Runners-Up,Third,Fourth
Brazil,5.0,2.0,2.0,2.0
Italy,4.0,2.0,1.0,1.0
Germany FR,3.0,3.0,1.0,1.0
Uruguay,2.0,0.0,0.0,3.0
Argentina,2.0,3.0,0.0,0.0
England,1.0,0.0,0.0,1.0
France,1.0,1.0,2.0,1.0
Spain,1.0,0.0,0.0,1.0
Germany,1.0,1.0,3.0,0.0
Netherlands,0.0,3.0,1.0,1.0


Trial using plotly to create graph of winners. 

Note: Germany FR is the Federal Republic of Germany which was commonly known as West Germany. |

In [84]:
basic_fig = go.Figure()
fig = px.bar(main_world_cup_info, x = "Winner")
fig

In [85]:
fig2 = px.bar(main_world_cup_info, x = "Runners-Up")
fig2

### Total Finishes for all World Cup Games
A graph showing amount of times teams were winners, runners-up, third or fourth. Oddly with this graph the only way I could get it to work is it pilled on top of itself, for example: Brazil has 5 wins, then they have come in 2nd twice, as well as come in 3rd and 4th twice.

In [99]:
fig3 = px.bar(format_finishes)
fig3

### Goals Scored by Country 

In [109]:
fig4 = px.bar(main_world_cup_info, x = "Country", y = "GoalsScored", barmode="group")
fig4

### Qualified Teams By Country

In [110]:
fig5 = px.bar(main_world_cup_info, x = "Country", y = "QualifiedTeams", barmode="group")
fig5

In [122]:
fig6 = px.pie(main_world_cup_info, values = "GoalsScored", names = "Country", title = "Percentage of Goals Scored by Country")
fig6

In [125]:
fig7 = px.pie(format_finishes, title = "Percentage of Goals Scored by Country")
fig7